In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import re

from tqdm import tqdm

## Obtain Website Datas

In [2]:
list_genres = [{'genre': 'Comedy','url': 'https://www.imdb.com/search/title/?genres=comedy&explore=title_type,genres&ref_=adv_nxt'},
                {'genre': 'Sci-Fi','url': 'https://www.imdb.com/search/title/?genres=sci-fi&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Horror','url': 'https://www.imdb.com/search/title/?genres=horror&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Romance','url': 'https://www.imdb.com/search/title/?genres=romance&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Action','url': 'https://www.imdb.com/search/title/?genres=action&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Thriller','url': 'https://www.imdb.com/search/title/?genres=thriller&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Drama','url': 'https://www.imdb.com/search/title/?genres=drama&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Mystery','url': 'https://www.imdb.com/search/title/?genres=mystery&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Crime','url': 'https://www.imdb.com/search/title/?genres=crime&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Animation','url': 'https://www.imdb.com/search/title/?genres=animation&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Adventure','url': 'https://www.imdb.com/search/title/?genres=adventure&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Fantasy','url': 'https://www.imdb.com/search/title/?genres=fantasy&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Comedy-romance','url': 'https://www.imdb.com/search/title/?genres=comedy,romance&explore=title_type,genres&ref_=adv_nxt'},
              {'genre': 'Action-comedy','url': 'https://www.imdb.com/search/title/?genres=action,comedy&explore=title_type,genres&ref_=adv_nxtt'},]
list_genres

[{'genre': 'Comedy',
  'url': 'https://www.imdb.com/search/title/?genres=comedy&explore=title_type,genres&ref_=adv_nxt'},
 {'genre': 'Sci-Fi',
  'url': 'https://www.imdb.com/search/title/?genres=sci-fi&explore=title_type,genres&ref_=adv_nxt'},
 {'genre': 'Horror',
  'url': 'https://www.imdb.com/search/title/?genres=horror&explore=title_type,genres&ref_=adv_nxt'},
 {'genre': 'Romance',
  'url': 'https://www.imdb.com/search/title/?genres=romance&explore=title_type,genres&ref_=adv_nxt'},
 {'genre': 'Action',
  'url': 'https://www.imdb.com/search/title/?genres=action&explore=title_type,genres&ref_=adv_nxt'},
 {'genre': 'Thriller',
  'url': 'https://www.imdb.com/search/title/?genres=thriller&explore=title_type,genres&ref_=adv_nxt'},
 {'genre': 'Drama',
  'url': 'https://www.imdb.com/search/title/?genres=drama&explore=title_type,genres&ref_=adv_nxt'},
 {'genre': 'Mystery',
  'url': 'https://www.imdb.com/search/title/?genres=mystery&explore=title_type,genres&ref_=adv_nxt'},
 {'genre': 'Crime'

Remarks: A dictionary for website genres is created.

Accessing each individual genre

In [3]:
input_url = list_genres[13]['url']
input_url

'https://www.imdb.com/search/title/?genres=action,comedy&explore=title_type,genres&ref_=adv_nxtt'

Remarks:
1. Able to access, each of 1st page in every genre

## Webscraping 

Strategy: write a webscraping code that allows the user to get the necessary information for data visualisation and machine learning.

The function below aims to webscrape movies on a single web page.
1. The function and pull out details of a movie into dictionary
2. Append the dictionary into a list
3. Repeats the process until all movie details have been scraped from a single webpage.

In [4]:
def movie_details():
# Obtain necesary information from each movie entry
    title_tag = movie[i].find('a')
    if title_tag is not None:
        title = title_tag.text.strip()
    else:
        title = None
    
    year_tag = movie[i].find('span',{'class':'lister-item-year text-muted unbold'})
    if year_tag is not None:
        year = year_tag.text
    else:
        year = None

    certificate_tag = movie[i].find('span',{'class':'certificate'})
    if certificate_tag is not None:
        certificate = certificate_tag.text
    else:
        certificate = None

    rating_tag = movie[i].find('strong')
    if rating_tag is not None:
        rating = rating_tag.text
    else:
        rating = None

    members = movie[i].find('p',{'class': ""}).text.strip().split('\n')
    actors_lst = [x.replace(', ','') for x in members]
    actors = actors_lst[1:]

    description_tag = movie[i].find_all('p',{'class': 'text-muted'})[1].text.strip()
    if description_tag is not None:
        description = description_tag
    else:
        description = None
    return {'Genre': list_genres[genre]['genre'],'Title': title,
            'Year_produced': year,'Certificate': certificate,'Ratings': rating,
            'Description': description,'Actors': actors}

Remarks: Function was created.

The code below aims to do the following:

1. Iterate through all the list of genres and obtain request and response for webscrapping
2. Using the function to scrape information and place it into a dictionary, and append it into a list
3. Finds the 'next page' button to access the next page
4. Enters the next page and obtain request for webscrapping
5. repeats steps 1 to 4 for a 1,000 pages in each genre.

In [5]:
headers = {'Accept-Language': 'en-US,en;q=0.9'}

movie_all = []
for genre in tqdm(range (len(list_genres))):
    html = requests.get(list_genres[genre]['url'],headers=headers, timeout = 60)
    raw = html.text
    soup = BeautifulSoup(raw, 'lxml')
    movie = soup.find_all('div', {'class': 'lister-item-content'})
    for i in range (len(movie)):
        movie_all.append(movie_details())
    
    next_button = soup.find('a', {'class': 'lister-page-next next-page'})['href']
    #scrape subsequent pages of genre
    for new in tqdm(range (1000)):
        html = requests.get('https://www.imdb.com/'+next_button, headers=headers, timeout = 60)
        raw = html.text
        soup = BeautifulSoup(raw, 'lxml')
        movie = soup.find_all('div', {'class': 'lister-item-content'})
        for i in range (len(movie)):
            movie_all.append(movie_details())
        next_button = soup.find('a', {'class': 'lister-page-next next-page'})['href']
        

100%|██████████| 14/14 [10:43:33<00:00, 2758.10s/it]


Remarks: webpage details has been scrapped.

## Dataframe

Create a data frame to store the scrapped details.

In [6]:
df = pd.DataFrame(movie_all, columns = ['Genre','Title','Year_produced','Certificate','Ratings', 'Description','Actors'])

In [7]:
df.head()

,Genre,Title,Year_produced,Certificate,Ratings,Description,Actors
0,Comedy,Beef,(2023– ),TV-MA,8.2,Two people let a road rage incident burrow int...,"[Steven Yeun, Ali Wong, Joseph Lee, Young Mazino]"
1,Comedy,Succession,(2018–2023),TV-MA,8.8,The Roy family is known for controlling the bi...,"[Nicholas Braun, Brian Cox, Kieran Culkin, Pet..."
2,Comedy,The Super Mario Bros. Movie,(2023),PG,7.3,The story of The Super Mario Bros. on their jo...,"[Aaron Horvath, Michael Jelenic, Pierre Leduc,..."
3,Comedy,Ted Lasso,(2020– ),TV-MA,8.8,American college football coach Ted Lasso head...,"[Jason Sudeikis, Brett Goldstein, Hannah Waddi..."
4,Comedy,Ghosted,(2023),PG-13,5.8,Cole falls head over heels for enigmatic Sadie...,"[Dexter Fletcher, | , Stars:, Chris Evans,..."


In [8]:
df.shape

(700698, 7)

Remarks: data has been transformed into a dataframe.

## Summary

In summary:

1. 14 different genres of movie/shows has been webscrapped from IMDb website
2. A total of 14,014 pages worth of movie details has been collected
3. A total of 700,698 movie details were collected.
4. 7 different types of features were collected: Genre, Title, Year Produced, Certificate of ratings, Description of movie/show and List of star actors.

In [9]:
df.to_csv('all_movies_02.csv', index=False)

Remarks: dataset has been exported.